In [19]:
from pymongo import MongoClient
import folium
import pandas as pd

In [20]:
def connectCollection(database, collection):
    client = MongoClient()
    db = client[database]
    coll = db[collection]
    return db, coll

In [21]:
db, coll = connectCollection('companies','companies')

In [22]:
def getLocation(comp,i):
    latitude = comp['offices'][i]['latitude']
    longitude = comp['offices'][i]['longitude']
    loc = {
        'type':'Point',
        'coordinates':[longitude, latitude]
    }
    return loc

In [23]:
def getName(comp,i):
    name = comp['name']
    return name

In [47]:
pipeline = [
    { "$unwind": "$offices"},
    {"$match":{ "$and": [  {"founded_year":{"$gt":2003}},
            {"funding_rounds.raised_amount":{"$gt":1000000}}, { "$or":[{"category_code":"web"},
            {"category_code":"software"},{"category_code":"games_video"},
            {"category_code":"hardware"},{"category_code":"mobile"},{"category_code":"music"},
            {"category_code":"photo_video"},{"category_code":"design"}]}] }}
#    {"$match":"}
    ]
results = list(coll.aggregate(pipeline))

In [51]:
name = []
city = []
country_code = []
longitude = []
latitude = []

for e in results:
    name.append(e["name"])
    city.append(e['offices']["city"])
    country_code.append(e['offices']["country_code"])
    for c in e["offices"].items():
        if c[0]=="longitude":
            longitude.append(c[1])
        elif c[0]=="latitude":
            latitude.append(c[1])

In [52]:
print(len(name),len(city),len(country_code),len(longitude),len(latitude))

1211 1211 1211 1211 1211


In [54]:
myData = {"name":name, "city":city, "country":country_code, "latitude":latitude, "longitude":longitude}

In [55]:
startup_df = pd.DataFrame(data=myData)

In [57]:
startup_df

,name,city,country,latitude,longitude
0,Wetpaint,Seattle,USA,47.603122,-122.333253
1,Wetpaint,New York,USA,40.723731,-73.996431
2,Geni,West Hollywood,USA,34.090368,-118.393064
3,Slacker,San Diego,USA,33.022176,-117.081406
4,Joost,New York,USA,40.746497,-74.009447
...,...,...,...,...,...
1206,Antix Labs,Berkshire,GBR,NaN,NaN
1207,Incuity Software,Mission Viejo,USA,33.664087,-117.632519
1208,Bump Technologies,Mountain View,USA,NaN,NaN
1209,Tiny Speck,Vancouver,CAN,49.282455,-123.109217


In [58]:
startup_clean = startup_df.dropna()

In [59]:
startup_clean

,name,city,country,latitude,longitude
0,Wetpaint,Seattle,USA,47.603122,-122.333253
1,Wetpaint,New York,USA,40.723731,-73.996431
2,Geni,West Hollywood,USA,34.090368,-118.393064
3,Slacker,San Diego,USA,33.022176,-117.081406
4,Joost,New York,USA,40.746497,-74.009447
...,...,...,...,...,...
1195,Clavis Technology,Cambridge,USA,42.362663,-71.084153
1197,Equiendo,Nangor Road,IRL,53.337074,-6.251891
1205,Continuity Engine,New Haven,USA,41.327918,-73.243552
1207,Incuity Software,Mission Viejo,USA,33.664087,-117.632519


In [60]:
import folium

In [ ]:
map_city=folium.Map(location=[40.7221,-73.9712], zoom_start=12)
for wendy in manhattanWendys:
    folium.Marker(wendy['location']['coordinates'][::-1],
                    radius=2,
                    icon=folium.Icon(icon='cloud',color='red'), 
                   ).add_to(map_city)
    
for wendy in manhattanWendys[:5]:
    folium.Marker(wendy['location']['coordinates'][::-1],
                    radius=2,
                    icon=folium.Icon(icon='cloud',color='blue'), 
                   ).add_to(map_city)